# Поиск токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

# Цели и задачи проекта

**Цели:**

Обучить модель классифицировать комментарии на позитивные и негативные.

Модель позволит магазину искать токсичные комментарии и отправлять их на модерацию.

**Задачи:**
1. Загрузим и подготовим данные.
2. Обучим разные модели.
3. Сделаем выводы.

# Описание данных

В нашем распоряжении набор данных с разметкой о токсичности правок.

Данные находятся в файле `/datasets/toxic_comments.csv`.

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Содержание

1  [Подготовка](#1)

*    1.1  [Очистка](#11)
*    1.2  [Лемматизация *WordNetLemmatizer*](#12)
*    1.3  [Лемматизация *WordNetLemmatizer + pos_tag*](#13)

2  [Обучение](#2)

*    2.1  [*LogisticRegression*](#21)
*    2.2  [*DecisionTreeClassifier*](#22)
*    2.3  [*LGBMClassifier*](#23)
*    2.4  [Сравнение](#24)

3  [Тестирование](#3)

4  [Общий вывод](#4)

<a name="1"></a>
## Подготовка

In [1]:
import pandas as pd
import numpy as np
import datetime

from IPython.display import display

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

import re

In [2]:
data = pd.read_csv(r'C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Мои проекты/GitHubRepositories/'
                 'YandexPracticum/11_ML_for_texts_search_for_toxic_comments/toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
display(data.sample(5))

,Unnamed: 0,text,toxic
158863,159022,My feeling is that it's a hoax — but that woul...,0
109594,109691,"""\nAs I suggested earlier, I brought this bloc...",0
116378,116477,"""\n\n Your submission at Articles for creation...",0
119714,119819,Claus \n\nClaus is on all the maps but not men...,0
36174,36216,queer ur gay i hope u read this,1


In [5]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


**Вывод**. 

В этом разделе были выполнены чтение и подготовка данных.

В результате выполнения задач этого раздела было выявлено следующее:
1. пропусков в данных нет;
2. типы данных соответствуют требованиям для последующей очистки и лемматизации комментариев;
3. целевой признак *toxic* имеет положительные значения для 10% комментариев (среднее значение *mean ~ 0.1*).

Таким образом, данные подготовлены для очистки и лемматизации комментариев.

<a name="11"></a>
### Очистка

In [6]:
def clear_text(text):
    
    '''переводит в нижний регистр, оставляет только латиницу, удаляет stop_words'''
    
    stop_words = set(nltk_stopwords.words('english'))
    text = text.lower()
    word_list = re.sub(r"[^a-z ]", ' ', text).split()
    word_notstop_list = [w for w in word_list if not w in stop_words]
    return ' '.join(word_notstop_list)

In [7]:
data['clean_text'] = data['text'].apply(clear_text)

In [8]:
display(data.sample(5))

,Unnamed: 0,text,toxic,clean_text
122519,122625,"oi \n\nwhy did you give me a last warning, why...",1,oi give last warning cunt
137363,137501,DryBones Palestine.jpg\nwhat happend to the Pa...,0,drybones palestine jpg happend palestianian th...
131623,131759,Keep it current \n\nWhen listing potential TNA...,0,keep current listing potential tna tc winners ...
122233,122338,Silencing references through IP blocks? You're...,1,silencing references ip blocks seriously pathetic
156762,156921,up\n\n (Talk) \n\n (Talk) \n\n (Talk) \n\nB...,0,talk talk talk blocked blanking wikipedia arti...


<a name="12"></a>
### Лемматизация *WordNetLemmatizer*

In [9]:
def lemm_text(text):
    
    '''Лемматизирует строку WordNetLemmatizer'''
    
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_text

In [10]:
beg_time = datetime.datetime.now()
data['wnl_text'] = data['clean_text'].apply(lemm_text)
data_lemm_time = (datetime.datetime.now()-beg_time).seconds

print(data_lemm_time)

17


<a name="13"></a>
### Лемматизация *WordNetLemmatizer + pos_tag*

In [11]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [12]:
def postag_lemm_text(text):
    
    '''Лемматизирует строку WordNetLemmatizer с учетом nltk.pos_tag'''
    
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemmatized_text

In [13]:
beg_time = datetime.datetime.now()
data['wnlpostag_text'] = data['clean_text'].apply(postag_lemm_text)
data_lemm_time = (datetime.datetime.now()-beg_time).seconds

print(data_lemm_time)

2270


In [14]:
# промежуточно сохраняем файл с очищенными и лемматизированными комментариями
'''
data.to_csv('data_lemm2.csv', index=False)
data = pd.read_csv('data_lemm2.csv')'''
display(data.sample(3).T)

,23053,51000,146121
Unnamed: 0,23073,51056,146277
text,"And on another note, what right does he have t...",(UTC)\n\nI added to ODP link because it includ...,"""\n\n A Chelsea-persona page sounds excellent ..."
toxic,0,0,0
clean_text,another note right send blocking threats state...,utc added odp link includes links urban explor...,chelsea persona page sounds excellent much res...
wnl_text,another note right send blocking threat state ...,utc added odp link includes link urban explora...,chelsea persona page sound excellent much resi...
wnlpostag_text,another note right send block threat state poi...,utc add odp link include link urban exploratio...,chelsea persona page sound excellent much resi...


**Вывод**. 

В этом разделе были выполнены следующие задачи:
1. комментарии очищены: буквы переведены в нижний регистр, оставлена только латиница, удалены стоп-слова;
2. комментарии лемматизированы без учёта части речи и с учётом части речи (POS-тегов).

Таким образом, данные подготовлены для обучения моделей.

<a name="2"></a>
## Обучение

In [15]:
kfold = KFold(n_splits=5, random_state=123, shuffle=True)

In [16]:
corpus = data['wnl_text']
corpus_2 = data['wnlpostag_text']

In [17]:
features_train, features_test, target_train, target_test = train_test_split(
    corpus, 
    data['toxic'].values, 
    test_size=0.2, stratify=data['toxic'].values, shuffle=True, random_state=123)

In [18]:
features_train_2, features_test_2, target_train_2, target_test_2 = train_test_split(
    corpus_2, 
    data['toxic'].values, 
    test_size=0.2, stratify=data['toxic'].values, shuffle=True, random_state=123)

In [19]:
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

In [20]:
count_tf_idf_2 = TfidfVectorizer()
tf_idf_train_2 = count_tf_idf_2.fit_transform(features_train_2)
tf_idf_test_2 = count_tf_idf_2.transform(features_test_2)

<a name="21"></a>
### *LogisticRegression*

In [21]:
beg_time = datetime.datetime.now()

model_1 = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=123)

model_1.mod = 'model_1'
model_1.name = 'LogisticRegression'
model_1.data = 'wnl_text'
model_1.f1 = cross_val_score(model_1, tf_idf_train, target_train, cv=kfold, scoring='f1')
model_1.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_1.f1.mean()), 
      'модель:', model_1.name, 
      'данные:', model_1.data, 
      'время работы модели:', model_1.time)

f1: 0.754 модель: LogisticRegression данные: wnl_text время работы модели: 6


- f1: 0.754
- модель: LogisticRegression 
- данные: wnl_text 
- время работы модели: 6

In [22]:
beg_time = datetime.datetime.now()

model_2 = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=123)

model_2.mod = 'model_2'
model_2.name = 'LogisticRegression'
model_2.data = 'wnlpostag_text'
model_2.f1 = cross_val_score(model_2, tf_idf_train_2, target_train_2, cv=kfold, scoring='f1')

model_2.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_2.f1.mean()), 
      'модель:', model_2.name, 
      'данные:', model_2.data, 
      'время работы модели:', model_2.time)

f1: 0.752 модель: LogisticRegression данные: wnlpostag_text время работы модели: 6


- f1: 0.752
- модель: LogisticRegression 
- данные: wnlpostag_text 
- время работы модели: 6

In [23]:
# from sklearn.model_selection import cross_validate

# model_1 = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=123)

# model_1.mod = 'model_1'
# model_1.name = 'LogisticRegression'
# model_1.data = 'wnl_text'
# model_1.f1 =  cross_validate(model_1, tf_idf_train, target_train, cv=kfold, scoring='f1')
# model_1.time = (datetime.datetime.now()-beg_time).seconds

# print('f1: %.3f' %(model_1.f1['test_score'].mean()), 
#       'модель:', model_1.name, 
#       'данные:', model_1.data, 
#       'время обучения fit_time: %.0f' %(model_1.f1['fit_time'].mean()),
#       'время предсказания score_time: %.0f' %(model_1.f1['score_time'].mean()))

f1: 0.754 модель: LogisticRegression данные: wnl_text время обучения fit_time: 1 время предсказания score_time: 0

<a name="22"></a>
### *DecisionTreeClassifier*

In [24]:
beg_time = datetime.datetime.now()

model_3 = DecisionTreeClassifier(class_weight='balanced', random_state=123)

model_3.mod = 'model_3'
model_3.name = 'DecisionTreeClassifier'
model_3.data = 'wnl_text'
model_3.f1 = cross_val_score(model_3, tf_idf_train, target_train, cv=kfold, scoring='f1')

model_3.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_3.f1.mean()), 
      'модель:', model_3.name, 
      'данные:', model_3.data, 
      'время работы модели:', model_3.time)

f1: 0.656 модель: DecisionTreeClassifier данные: wnl_text время работы модели: 554


- f1: 0.656
- модель: DecisionTreeClassifier 
- данные: wnl_text 
- время работы модели: 511

In [25]:
beg_time = datetime.datetime.now()

model_4 = DecisionTreeClassifier(class_weight='balanced', random_state=123)

model_4.mod = 'model_4'
model_4.name = 'DecisionTreeClassifier'
model_4.data = 'wnlpostag_text'
model_4.f1 = cross_val_score(model_4, tf_idf_train_2, target_train_2, cv=kfold, scoring='f1')

model_4.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_4.f1.mean()), 
      'модель:', model_4.name, 
      'данные:', model_4.data, 
      'время работы модели:', model_4.time)

f1: 0.659 модель: DecisionTreeClassifier данные: wnlpostag_text время работы модели: 446


- f1: 0.659
- модель: DecisionTreeClassifier 
- данные: wnlpostag_text 
- время работы модели: 494

<a name="23"></a>
### *LGBMClassifier*

In [26]:
beg_time = datetime.datetime.now()

model_5 = LGBMClassifier(n_estimators=50, class_weight='balanced', boosting_type='gbdt', 
                         objective='binary', random_state=123)

model_5.mod = 'model_5'
model_5.name = 'LGBMClassifier 50'
model_5.data = 'wnl_text'
model_5.f1 = cross_val_score(model_5, tf_idf_train, target_train, cv=kfold, scoring='f1')

model_5.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_5.f1.mean()), 
      'модель:', model_5.name, 
      'данные:', model_5.data, 
      'время работы модели:', model_5.time)

f1: 0.723 модель: LGBMClassifier 50 данные: wnl_text время работы модели: 61


- f1: 0.723
- модель: LGBMClassifier 
- n_estimators = 50
- данные: wnl_text  
- время работы модели: 74

In [27]:
beg_time = datetime.datetime.now()

model_6 = LGBMClassifier(n_estimators=50, class_weight='balanced', boosting_type='gbdt', 
                         objective='binary', random_state=123)

model_6.mod = 'model_6'
model_6.name = 'LGBMClassifier 50'
model_6.data = 'wnlpostag_text'
model_6.f1 = cross_val_score(model_6, tf_idf_train_2, target_train_2, cv=kfold, scoring='f1')

model_6.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_6.f1.mean()), 
      'модель:', model_6.name, 
      'данные:', model_6.data, 
      'время работы модели:', model_6.time)

f1: 0.727 модель: LGBMClassifier 50 данные: wnlpostag_text время работы модели: 56


- f1: 0.727
- модель: LGBMClassifier 
- n_estimators = 50
- данные: wnlpostag_text  
- время работы модели: 61

In [28]:
beg_time = datetime.datetime.now()

model_7 = LGBMClassifier(n_estimators=500, class_weight='balanced', boosting_type='gbdt', 
                         objective='binary', random_state=123)

model_7.mod = 'model_7'
model_7.name = 'LGBMClassifier 500'
model_7.data = 'wnl_text'
model_7.f1 = cross_val_score(model_7, tf_idf_train, target_train, cv=kfold, scoring='f1')

model_7.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_7.f1.mean()), 
      'модель:', model_7.name, 
      'данные:', model_7.data, 
      'время работы модели:', model_7.time)

f1: 0.764 модель: LGBMClassifier 500 данные: wnl_text время работы модели: 329


- f1: 0.764
- модель: LGBMClassifier 
- n_estimators = 500
- данные: wnl_text  
- время работы модели: 358

In [29]:
beg_time = datetime.datetime.now()

model_8 = LGBMClassifier(n_estimators=500, class_weight='balanced', boosting_type='gbdt', 
                         objective='binary', random_state=123)

model_8.mod = 'model_8'
model_8.name = 'LGBMClassifier 500'
model_8.data = 'wnlpostag_text'
model_8.f1 = cross_val_score(model_5, tf_idf_train_2, target_train_2, cv=kfold, scoring='f1')

model_8.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_8.f1.mean()), 
      'модель:', model_8.name, 
      'данные:', model_8.data, 
      'время работы модели:', model_8.time)

f1: 0.727 модель: LGBMClassifier 500 данные: wnlpostag_text время работы модели: 55


- f1: 0.727
- модель: LGBMClassifier 
- n_estimators = 500
- данные: wnlpostag_text  
- время работы модели: 57

<a name="24"></a>
### Сравнение

In [30]:
model_list = [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8]

In [31]:
a={}
for i in model_list:
    b={}    
    b['model']=i.name
    b['data']=i.data
    b['f1_score']=i.f1.mean()
    b['cross_val_time']=i.time
    a[i.mod] = b

final_table = pd.DataFrame(a)

In [32]:
display(final_table.T)

,model,data,f1_score,cross_val_time
model_1,LogisticRegression,wnl_text,0.753929,6
model_2,LogisticRegression,wnlpostag_text,0.752032,6
model_3,DecisionTreeClassifier,wnl_text,0.656351,554
model_4,DecisionTreeClassifier,wnlpostag_text,0.658628,446
model_5,LGBMClassifier 50,wnl_text,0.722983,61
model_6,LGBMClassifier 50,wnlpostag_text,0.726778,56
model_7,LGBMClassifier 500,wnl_text,0.763801,329
model_8,LGBMClassifier 500,wnlpostag_text,0.726778,55


**Вывод**. 

В этом разделе были выполнены следующие задачи:
- обучены 8 моделей (*LogisticRegression*, *DecisionTreeClassifier* и *LGBMClassifier* с количеством деревьев 50 и 500) без учёта части речи и с учётом части речи (POS-тегов);

В результате выполнения задач этого раздела было выявлено следующее:
- в качестве лучшей модели (с наибольшим значением метрики *F1*) выбрана:
   - *LGBMClassifier* на данных, лемматизированных без учета части речи, и с количеством деревьев n_estimators=500.

<a name="3"></a>
## Тестирование

In [33]:
model_7.fit(tf_idf_train, target_train)
model_7.predicted = model_7.predict(tf_idf_test)
model_7.test_f1 = f1_score(target_test, model_7.predicted)
print(model_7.test_f1)

0.7722490759169746


- f1: 0.772
- модель: LGBMClassifier 
- n_estimators = 500
- данные: wnl_text

**Вывод**. 

В этом разделе были выполнены следующие задачи:
- протестирована лучшая модель:
   - *LGBMClassifier* на данных, лемматизированных без учета части речи, и с количеством деревьев n_estimators=500.
   
В результате выполнения задач этого раздела было выявлено следующее:
- в выбранной модели значение метрики качества *F1* превышает 0.75, как и изначально требовалось по условию задачи проекта.

<a name="4"></a>
## Общий вывод

Проведено исследование с целью обучить модель классифицировать комментарии на позитивные и негативные.

Входные данные - набор данных с разметкой о токсичности правок.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.

В результате исследования удалось получить следующие результаты **на обучающей выборке**:


**1. Модель *LogisticRegression*:**

   - данные, лемматизированые без учёта части речи (*wnl_text*):
      - *F1* = 0.754
      - время работы модели: 6 с


   - данные, лемматизированые с учётом части речи - POS-тегов (*wnlpostag_text*):
      - *F1* = 0.752
      - время работы модели: 6 с


**2. Модель *DecisionTreeClassifier*:**

   - данные, лемматизированые без учёта части речи (*wnl_text*):
      - *F1* = 0.656
      - время работы модели: 511 с


   - данные, лемматизированые с учётом части речи - POS-тегов (*wnlpostag_text*):
      - *F1* = 0.659
      - время работы модели: 494 с


**3. Модель *LGBMClassifier*:**

- количество деревьев *n_estimators* = 50:

   - данные, лемматизированые без учёта части речи (*wnl_text*):
      - *F1* = 0.723
      - время работы модели: 74 с
   - данные, лемматизированые с учётом части речи - POS-тегов (*wnlpostag_text*):
      - *F1* = 0.727
      - время работы модели: 61 с


- количество деревьев *n_estimators* = 500:

   - данные, лемматизированые без учёта части речи (*wnl_text*):
      - *F1* = 0.764
      - время работы модели: 358 с
   - данные, лемматизированые с учётом части речи - POS-тегов (*wnlpostag_text*):
      - *F1* = 0.727
      - время работы модели: 57 с
      
      
      
Исходя из полученных результатов, можно сделать следующие **выводы**:


1. В качестве лучшей модели (с наибольшим значением метрики *F1*) выбрана:
   - *LGBMClassifier* на данных, лемматизированных без учета части речи (*wnl_text*), и с количеством деревьев *n_estimators*=500.
   
   
2. Значение метрики *F1* = 0.772 для модели *LGBMClassifier* **на тестовой выборке**.
   
   
3. В выбранной модели значение метрики качества *F1* превышает 0.75, как и изначально требовалось по условию задачи проекта.
   
   
**Общие рекомендации:**

Магазину можно рекомендовать использовать полученную модель ***LGBMClassifier*** в качестве инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.

**Чек-лист проверки**

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны